In [0]:
%run "../../../config/setup"

In [0]:
CREATE OR REPLACE VIEW transactions_customer_dedup AS
  SELECT 
    max(transaction_id) AS transaction_id, 
    customer_id,
    max(transaction_date) AS transaction_date, 
    max(total_amount) AS total_amount,
    max(source_file) AS source_file, 
    max(ingest_ts) AS ingest_ts 
  FROM transactions_bronze
  WHERE customer_id IS NOT NULL 
  GROUP BY customer_id
  

In [0]:
GRANT SELECT ON VIEW transactions_customer_dedup TO devs;

In [0]:
CREATE OR REPLACE VIEW transactions_dedup AS
  SELECT 
    transaction_id,
    max(customer_id) AS customer_id,
    max(transaction_date) AS transaction_date, 
    max(total_amount) AS total_amount,
    max(source_file) AS source_file, 
    max(ingest_ts) AS ingest_ts 
  FROM transactions_customer_dedup
  WHERE transaction_id IS NOT NULL 
  GROUP BY transaction_id
  

In [0]:
GRANT SELECT ON VIEW transactions_dedup TO devs;

In [0]:
CREATE OR REPLACE TABLE transactions_silver AS
  SELECT 
    transaction_id,
    CAST(customer_id AS INT) AS customer_id,
    transaction_date, 
    total_amount
  FROM transactions_dedup
  

In [0]:
GRANT SELECT ON TABLE transactions_silver TO devs;